In [1]:
import openpyxl
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing  import LabelEncoder
from datetime import datetime



# 데이터 가져오기

In [238]:
df_active = pd.read_excel("./data/1_Active_MS_Business_Info.xlsx", sheet_name = 0)
df_active_record = pd.read_excel("./data/1_Active_MS_Business_Info.xlsx", sheet_name = 2)
df_inactive = pd.read_excel("./data/2_Inactive_MS_Business_Info.xlsx", sheet_name = 0)
df_inactive_record = pd.read_excel("./data/2_Inactive_MS_Business_Info.xlsx", sheet_name = 1)

In [240]:
# 기업명 컬럼명 변경
df_active = df_active.rename(columns={'CMP_NM1':'CMP_NM'})

In [241]:
df_active.isnull().sum()

BIZ_NO                 0
CMP_PFIX_NM        10402
CMP_NM                 0
CMP_SFIX_NM        19813
CMP_ENM               17
BZ_TYP                 0
CMP_SCL                0
PBCO_GB                0
PSN_CORP_GB            0
HDOF_BR_GB             0
FR_IVST_CORP_YN        0
VENT_YN                0
LIST_CD            28893
IND_CD_ORDR           10
IND_CD1               11
IND_CD2            27782
IND_CD3            28719
PB_ORG_TYP             0
MDSCO_PRTC_YN          0
HDOF_CMP_CD        26075
ESTB_DATE           1074
ESTB_GB                0
LIST_DATE          28823
MN_BIZ_CONT          163
NATN_NM            27337
HOMEPAGE_URL       11228
CEO_NM                 1
EMP_CNT              734
dtype: int64

# 데이터 전처리

In [242]:
#사용할 컬럼들
select_columns = ['BIZ_NO','CMP_NM', 'CEO_NM', 'BZ_TYP',  'HDOF_BR_GB', 'FR_IVST_CORP_YN', 'VENT_YN', 'LIST_CD', 'LIST_DATE', 'MDSCO_PRTC_YN', 'ESTB_DATE', 'ESTB_GB', 'NATN_NM', 'EMP_CNT']


In [243]:
df_active_select = df_active.loc[:, select_columns]
df_inactive_select = df_inactive.loc[:, select_columns]

In [244]:
def list_date_calc(series):

    # 결과 값을 담을 리스트 year_category 초기화
    year_category = []

    # for 반복문을 통해 판다스 시리즈 내의 각 날짜를 순회
    for date in series:

        # 날짜의 0번째 인덱스부터 3번째 인덱스까지 슬라이싱하여 정수형(Integer)으로 변환 후 변수 year에 할당
        year = int(date[0 : 4])

        # 각 설립연도의 조건에 따라 범주화
        if year < 1970: year_category.append("1970년 이전")
        elif year < 1980: year_category.append("1970년대")
        elif year < 1990: year_category.append("1980년대")
        elif year < 2000: year_category.append("1990년대")
        elif year < 2005: year_category.append("2000년대 전기")
        elif year < 2010: year_category.append("2000년대 후기")
        elif year < 2015: year_category.append("2010년대 전기")
        elif year < 2020: year_category.append("2010년대 후기")
        elif year < 2025: year_category.append("2020년대 전기")
        else: year_category.append("해당 없음")

    # 결과 값 반환
    return year_category


# 업력 구간화 함수

def estb_date_calc(series):

    # 결과 값을 담을 리스트 year_category 초기화
    year_category = []

    # for 반복문을 통해 판다스 시리즈 내의 각 날짜를 순회
    for date in series:

        # 날짜의 0번째 인덱스부터 3번째 인덱스까지 슬라이싱하여 정수형(Integer)으로 변환 후 변수 year에 할당
        year = int(date[0 : 4])

        # 각 설립연도의 조건에 따라 범주화
        if year < 1970: year_category.append("1970년 이전")
        elif year < 1980: year_category.append("1970년대")
        elif year < 1990: year_category.append("1980년대")
        elif year < 2000: year_category.append("1990년대")
        elif year < 2005: year_category.append("2000년대 전기")
        elif year < 2010: year_category.append("2000년대 후기")
        elif year < 2015: year_category.append("2010년대 전기")
        elif year < 2020: year_category.append("2010년대 후기")
        elif year < 2025: year_category.append("2020년대 전기")
        else: year_category.append("해당 없음")

    # 결과 값 반환
    return year_category


# 직원수 구간화
def get_category(EMP_CNT):
    EMP_cat = ''
    if EMP_CNT == 0 and -1: EMP_cat = 'NaN&zero'
    elif EMP_CNT <= 10: EMP_cat = '1~10'
    elif EMP_CNT <= 50: EMP_cat = '10~50'
    elif EMP_CNT <= 300: EMP_cat = '50~300'
    elif EMP_CNT > 300: EMP_cat = '300~'
    return EMP_cat


In [245]:
# 데이터 전처리 함수

def feature_engineering(df):

    # 대표자명
    df['CEO_NM'] = df['CEO_NM'].str.replace("외\s*[0-9]+명", "")



    #레이블 인코딩
    label_encode_features = ['FR_IVST_CORP_YN' , 'VENT_YN', 'MDSCO_PRTC_YN', 'NATN_NM', 'BZ_TYP']

    for feature in label_encode_features:
        le = LabelEncoder()
        le = le.fit(df[feature])
        df[feature] = le.transform(df[feature])

    # HDOF_BR_GB 본점지점구분 1.본점 2.지점
    # 따로 처리x



    #'LIST_CD' 상장여부로 0, 1
    df.LIST_CD.fillna(0, inplace=True)
    df['LIST_CD'].loc[df['LIST_CD']!=0]=1



    # 'LIST_DATE' 상장날짜 구간화
    df.LIST_DATE.fillna(99999999, inplace=True)
    df["LIST_DATE"] = df["LIST_DATE"].astype(str)
    # list_date_calc 실행한 결과를 다시 LIST_DATE 컬럼에 반영
    df["LIST_DATE"] = list_date_calc(df["LIST_DATE"])

    list_mapping = {"1970년 이전":0, "1970년대":1, "1980년대":2, "1990년대":3, "2000년대 전기":4, "2000년대 후기":5, "2010년대 전기":6, "2010년대 후기":7, "2020년대 전기":8, "해당 없음":9 }
    df['LIST_DATE'] = df['LIST_DATE'].map(list_mapping)
     


    # 'ESTB_DATE' 설립날짜 -> 업력 구간화, 인코딩
    df.ESTB_DATE.fillna(99999999, inplace=True)
    # 열의 값을 str으로 변경
    df["ESTB_DATE"] = df["ESTB_DATE"].astype(str)
    # estb_date_calc() 함수를 실행한 결과를 "ESTB_DATE_CAT" 열을 새로 만들어 기존 데이터프레임에 추가
    df["ESTB_DATE_CAT"] = estb_date_calc(df["ESTB_DATE"])
    
    #업력 인코딩
    estb_mapping = {"1970년 이전":0, "1970년대":1, "1980년대":2, "1990년대":3, "2000년대 전기":4, "2000년대 후기":5, "2010년대 전기":6, "2010년대 후기":7, "2020년대 전기":8, "해당 없음":9 }
    df['ESTB'] = df['ESTB_DATE_CAT'].map(estb_mapping)
    # 필요없는 컬럼 drop
    df = df.drop(['ESTB_DATE','ESTB_DATE_CAT'], axis=1)


    # 'ESTB_GB' 설립구분
     
     
    # 'EMP_CNT' 직원수 구간화
    df.EMP_CNT.fillna(-1, inplace=True)
    df['EMP_cat']=df['EMP_CNT'].apply(lambda x : get_category(x))

    # 직원수 범주화
    emp_mapping = {'NaN&zero':0, '1~10':1, '10~50':2, '50~300':3, '300~':4}
    df['EMP_cat'] = df['EMP_cat'].map(emp_mapping)
    


    return df
    

In [246]:
df_active_select = feature_engineering(df_active_select)
df_inactive_select = feature_engineering(df_inactive_select)

C:\Users\bin\AppData\Local\Temp\ipykernel_5604\865351939.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  df['CEO_NM'] = df['CEO_NM'].str.replace("외\s*[0-9]+명", "")
C:\Users\bin\AppData\Local\Temp\ipykernel_5604\865351939.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['LIST_CD'].loc[df['LIST_CD']!=0]=1
C:\Users\bin\AppData\Local\Temp\ipykernel_5604\865351939.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  df['CEO_NM'] = df['CEO_NM'].str.replace("외\s*[0-9]+명", "")
C:\Users\bin\AppData\Local\Temp\ipykernel_5604\865351939.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-do

In [247]:
#이상치 처리 - 직원수가 16310명인 회사 BIZ_NO 6038138153 광평건설
df_inactive_select =df_inactive_select.drop([204])
# index 재처리
df_inactive_select.reset_index(inplace=True)
df_inactive_select = df_inactive_select.drop('index', axis=1)
# 직원수 컬럼 drop
df_inactive_select = df_inactive_select.drop('EMP_CNT', axis=1)
df_active_select = df_active_select.drop('EMP_CNT', axis=1)

In [252]:
df_active_select.loc[df_active_select['CEO_NM'].isnull()]

,BIZ_NO,CMP_NM,CEO_NM,BZ_TYP,HDOF_BR_GB,FR_IVST_CORP_YN,VENT_YN,LIST_CD,LIST_DATE,MDSCO_PRTC_YN,ESTB_GB,NATN_NM,ESTB,EMP_cat
28969,1678100791,오송2단지1상가개발,NaN,4,1,0,0,0,9,0,1,63,7,1


In [250]:
df_inactive_select.loc[df_inactive_select['CEO_NM'].isnull()]

,BIZ_NO,CMP_NM,CEO_NM,BZ_TYP,HDOF_BR_GB,FR_IVST_CORP_YN,VENT_YN,LIST_CD,LIST_DATE,MDSCO_PRTC_YN,ESTB_GB,NATN_NM,ESTB,EMP_cat
6605,6618800502,알엠피플러스,NaN,0,1,0,0,0,9,0,7,27,9,0


In [253]:
# CEO_NM 결측치 active 1개, inactive 1개 - 'NaN'으로 채워줌
df_active_select['CEO_NM'].fillna('NaN', inplace=True)
df_inactive_select['CEO_NM'].fillna('NaN', inplace=True)

## 휴폐업이력sheet 컬럼 추가
- 휴폐업이력 유무(close_y_n)와 휴폐업횟수(close_count) 컬럼 만들기

In [255]:

def close_record(df):

    df.STAT_OCR_DATE.fillna(99999999, inplace=True)
    
    #close_y_n : 휴폐업 유무 컬럼
    df['close_y_n']=1

    #BIZ_NO로 set인덱스
    df=df.set_index('BIZ_NO')

    #휴폐업 조건으로 필터링한 새로운 컬럼 - 휴폐업이력 있는 기업만 들어있는 df 만들기
    close_condition = (df.CLSBZ_GB==1) | (df.CLSBZ_GB==2) | (df.CLSBZ_GB==3)
    condition_df_record = df[close_condition]

    #휴폐업 횟수 구하기
    condition_df_record = condition_df_record.groupby('BIZ_NO').sum()

    # 휴폐업 횟수 컬럼 추가
    df['close_count'] = condition_df_record['close_y_n']

    #index reset
    df = df.reset_index()

    #중복인덱스 삭제하기 - 열지정하여 중복제거
    df= df.drop_duplicates(['BIZ_NO']).reset_index(drop=True)

    return df

In [256]:
df_active_record = close_record(df_active_record)
df_inactive_record = close_record(df_inactive_record)
df_active_record.close_count.fillna(0, inplace=True)

### DF Join
- BIZ_NO 기준으로
- df_active_select + df_active_record
- df_rest_select + df_rest_record

In [257]:
#액티브 교집합(inner join) (362개)
df_active_innerjoin = pd.merge(df_active_select, df_active_record, left_on='BIZ_NO',right_on='BIZ_NO', how='inner')
df_active_innerjoin = df_active_innerjoin[['BIZ_NO', 'close_y_n', 'close_count']]

#휴폐업 기업개요와 휴폐업이력의 교집합(6693)
df_inactive_innerjoin = pd.merge(df_inactive_select, df_inactive_record,left_on='BIZ_NO',right_on='BIZ_NO', how='inner')
df_inactive_innerjoin = df_inactive_innerjoin[['BIZ_NO', 'close_y_n', 'close_count']]

#휴폐업 기업개요(6739=유니크), 휴폐업이력(6840)(유니크6746)

In [258]:
# 본 데이터프레임에 교집합innerjoin 데이터프레임 결합

df_active_join = pd.merge(df_active_select, df_active_innerjoin ,left_on='BIZ_NO',right_on='BIZ_NO', how='outer')
df_inactive_join = pd.merge(df_inactive_select, df_inactive_innerjoin ,left_on='BIZ_NO',right_on='BIZ_NO', how='outer')

In [259]:
# 널값 채우기
df_active_join = df_active_join.fillna(0)

In [260]:
df_active_join[['close_y_n', 'close_count']] = df_active_join[['close_y_n', 'close_count']].astype('int')

# 외부데이터


### 1) NICE
- (기업주소, 산업등급)
- nice 홈페이지 사업자번호 조회

In [261]:
df_nice_active = pd.read_excel("./외부data/act_company_information.xls", sheet_name = 0)
df_nice_inactive = pd.read_excel("./외부data/rest_company_information.xls", sheet_name = 0)

In [262]:
# 주소 비어있는 행 하나 (BIZ_NO : 5108119835)
df_nice_active[df_nice_active['address'].isnull()]

,BIZ_NO,grade,2021,2020,2019,address
13696,5108119835,5,"1,216,307","964,976","1,017,349",NaN


In [263]:
df_nice_active.address.fillna('NaN', inplace=True)

#### 전처리
- 1. 기업입지 도심vs부도심 -> 라벨링
- 2. grade는 그냥 사용
- 3. (매출의 증감폭)

In [264]:
city_df = pd.read_excel("./외부data/용도지역(시군구)-비도시지역_2021.xlsx", sheet_name = 0, index_col=[0,1,2])

c:\Users\bin\anaconda3\envs\numble\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [265]:
city_df.reset_index(drop=False, inplace=True)
city_df = pd.DataFrame(city_df.iloc[4:, :3])
city_df.columns=['시', '군,구', '2021']
city_df.drop(city_df.loc[city_df['군,구']=='소계'].index, inplace=True)
city_df['city'] = 0
city_df = city_df.reset_index()
city_df.drop('index', inplace=True, axis=1)

city_condition = city_df['2021']==0
city_df['city'].loc[city_condition]=1

C:\Users\bin\AppData\Local\Temp\ipykernel_5604\3754510174.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  city_df['city'].loc[city_condition]=1


In [266]:
# 도심(city) 컬럼 추가

df_nice_active['city'] = 0
df_nice_inactive['city'] = 0

city_index = city_df.loc[city_df['city']==1].index
city_list = city_df['군,구'][city_index].to_list()

#도시인 군,구들만 리스트 만들어서 그 안에 있으면 값 1주기
for df in [df_nice_active, df_nice_inactive] :
    for city in city_list:
        city_condition = df['address'].str.contains(city)
        df['city'].loc[city_condition]=1

C:\Users\bin\AppData\Local\Temp\ipykernel_5604\2511129021.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['city'].loc[city_condition]=1


In [267]:
# 필요한 컬럼만 선택
select_columns_ex = ['BIZ_NO', 'grade', 'city']

df_nice_active = df_nice_active[select_columns_ex]
df_nice_inactive = df_nice_inactive[select_columns_ex]

In [268]:
# 직원수 16000이었던 광평건설 drop
df_nice_inactive= df_nice_inactive.drop([195])

In [269]:
df_nice_inactive

,BIZ_NO,grade,city
0,1388101749,6,1
1,1068179240,6,1
2,2248114507,6,0
3,1168132785,5,1
4,6178132798,7,1
...,...,...,...
6010,1338801710,6,1
6011,5678101364,6,1
6012,3898602225,6,0
6013,1738102533,5,1


#### DF Join
- BIZ_NO 기준으로
- df_active_join + df_nice_active => active_df
- df_inactive_join + df_nice_inactive => inactive_df

In [270]:
active_df = pd.merge(df_active_join, df_nice_active, left_on='BIZ_NO', right_on='BIZ_NO', how='outer')
inactive_df = pd.merge(df_inactive_join, df_nice_inactive, left_on='BIZ_NO', right_on='BIZ_NO', how='outer')

In [271]:
active_df

,BIZ_NO,CMP_NM,CEO_NM,BZ_TYP,HDOF_BR_GB,FR_IVST_CORP_YN,VENT_YN,LIST_CD,LIST_DATE,MDSCO_PRTC_YN,ESTB_GB,NATN_NM,ESTB,EMP_cat,close_y_n,close_count,grade,city
0,1138111739,한일가스산업,홍순철,4,1,0,0,0,9,0,1,63,0,2,0,0,5.0,1.0
1,2078132193,엠피아이,이스테판상수,4,1,0,0,0,9,0,1,63,3,2,0,0,6.0,1.0
2,1318167565,도영운수,김명화,4,1,0,0,0,9,0,1,63,4,3,0,0,6.0,1.0
3,5048128251,중원산업,김명구,4,1,0,0,0,9,0,1,63,3,3,0,0,6.0,0.0
4,1098164260,카타나골프,김홍득,4,1,0,0,0,9,0,1,63,4,1,0,0,6.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28977,8048101453,제이앤인베스트먼트,정경태,4,1,0,0,0,9,0,1,63,7,0,0,0,NaN,NaN
28978,8028701430,디앤엠개발,송성현,4,1,0,0,0,9,0,1,63,8,1,0,0,6.0,0.0
28979,4268501421,마스비 건축연구소,김태남,4,2,0,0,0,9,0,1,63,3,3,0,0,NaN,NaN
28980,6518502462,건설레미콘의정부공장,이미성,4,2,0,0,0,9,0,1,63,8,2,0,0,NaN,NaN


In [272]:
active_df.isnull().sum()

BIZ_NO                0
CMP_NM                0
CEO_NM                0
BZ_TYP                0
HDOF_BR_GB            0
FR_IVST_CORP_YN       0
VENT_YN               0
LIST_CD               0
LIST_DATE             0
MDSCO_PRTC_YN         0
ESTB_GB               0
NATN_NM               0
ESTB                  0
EMP_cat               0
close_y_n             0
close_count           0
grade              2206
city               2206
dtype: int64

In [273]:
inactive_df.isnull().sum()

BIZ_NO               0
CMP_NM               0
CEO_NM               0
BZ_TYP               0
HDOF_BR_GB           0
FR_IVST_CORP_YN      0
VENT_YN              0
LIST_CD              0
LIST_DATE            0
MDSCO_PRTC_YN        0
ESTB_GB              0
NATN_NM              0
ESTB                 0
EMP_cat              0
close_y_n            0
close_count          0
grade              724
city               724
dtype: int64

In [274]:
active_df['city'].value_counts()

1.0    15576
0.0    11200
Name: city, dtype: int64

In [275]:
inactive_df['city'].value_counts()

1.0    3552
0.0    2462
Name: city, dtype: int64

1.
active_df 의 grade, city 널 값 2천개
inactive_df의 grade, city 700개
=> 어떤 값으로 채울지 고민 (머신러닝으로 예측해서 넣기? bike참조)

grade와 city(도심)

### 2) INNO_BIZ

In [422]:
inno_biz_df = pd.read_excel('./외부data/inno_biz.xlsx', header=2)
inno_biz_df = inno_biz_df.drop('번호', axis=1)

In [412]:
# '에이에스이'(innobiz) in '에이에스이 칠곡지점'(active)

In [423]:
inno_biz_df['업체명'] = inno_biz_df['업체명'].str.replace("\([^\)]+", "")
inno_biz_df['업체명'] = inno_biz_df['업체명'].str.replace("\)", "")
inno_biz_df['업체명'] = inno_biz_df['업체명'].str.replace(" *주식회사 *", "")



C:\Users\bin\AppData\Local\Temp\ipykernel_5604\3841284521.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  inno_biz_df['업체명'] = inno_biz_df['업체명'].str.replace("\([^\)]+", "")
C:\Users\bin\AppData\Local\Temp\ipykernel_5604\3841284521.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  inno_biz_df['업체명'] = inno_biz_df['업체명'].str.replace("\)", "")
C:\Users\bin\AppData\Local\Temp\ipykernel_5604\3841284521.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  inno_biz_df['업체명'] = inno_biz_df['업체명'].str.replace(" *주식회사 *", "")


In [427]:
active_df['INNO_BIZ']=0
inactive_df['INNO_BIZ']=0

In [428]:
inno_biz_df.loc[inno_biz_df['업체명'].str.contains('\(')]


,업체명,대표자,지역,평가지표,관련업종,인증만료일


In [429]:
active_df['INNO_BIZ'].value_counts()

0    28982
Name: INNO_BIZ, dtype: int64

In [430]:
inactive_df['INNO_BIZ'].value_counts()

0    6738
Name: INNO_BIZ, dtype: int64

In [431]:
inno_list = inno_biz_df['업체명'].tolist()

# 1. 기업명으로 한 번 거르기
for df in [active_df, inactive_df]:
    for inno in inno_list:
        condition = df['CMP_NM'].str.contains(inno)
        df['INNO_BIZ'].loc[condition]=1

C:\Users\bin\AppData\Local\Temp\ipykernel_5604\873086845.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['INNO_BIZ'].loc[condition]=1


In [285]:
# 2. INNO_BIZ가 1인 값들만 가져와서 새로운 df만들기
active_inno_df = active_df.loc[active_df['INNO_BIZ']==1]

# 대표자명만 리스트 만들기
active_inno_list = active_inno_df['CEO_NM'].tolist()

In [371]:
# inno_biz_df 에 대표자 리스트 이름 해당값만 남겨서 다시 df만들기
new_inno_index=[]
for ac in active_inno_list:
    condition = inno_biz_df['대표자'].str.contains(ac)
    try:
        index = inno_biz_df.loc[condition].index[0]
        new_inno_index.append(index)
    except:
        pass

In [373]:
len(new_inno_index)

8736

In [379]:
# 대표자명이 동명이인이면 동명이인 수 만큼 해당 기업 정보가 중복으로 리스트에 들어감
# -> 중복제거해주기
new_inno_biz=inno_biz_df.iloc[new_inno_index,:]
new_inno_biz = new_inno_biz.drop_duplicates(['업체명']).reset_index(drop=True)

In [380]:
new_inno_biz

,업체명,대표자,지역,평가지표,관련업종,인증만료일
0,버텍스아이디,홍순철,서울 송파구,SW,정보통신,2023-11-17
1,에스더전자주식회사,김명화,서울 금천구,제조업,전기전자,2023-08-17
2,형제정공,김명구,강원 원주시,제조업,기계금속,2024-07-19
3,세풍폴리머,김종만,서울 금천구,제조업,화학,2024-12-10
4,코팅코리아,최규호,전북 김제시,제조업,기계금속,2022-10-13
...,...,...,...,...,...,...
5364,케이알씨주식회사,최영범,대구 달성군,제조업,건설,2024-10-04
5365,주식회사뉴빌리티,이상민,서울 성동구,SW,서비스,2025-06-06
5366,코리아베르톨드,이진철,경기 성남시,제조업,기계금속,2023-01-13
5367,비스텔,최운규,서울 서초구,SW,S/W,2023-12-17


In [381]:
new_inno_biz_list = new_inno_biz['대표자'].tolist()
len(new_inno_biz_list)

5369

In [ ]:
for name in active_inno_list:
    if name in new_inno_biz_list:
        # 각각의 df에서 대표자이름으로 index구해서 해당 index의 회사이름 가져와서 같다면 최종리스트에 추가
        




    if (active_inno_df['CEO_NM']== name) and (new_inno_biz['대표자']==name):
        

In [382]:
active_inno_df['CMP_NM'].loc[active_inno_df['CEO_NM']=='홍순철'].tolist()

['한일가스산업', '나라삼양감속기']

In [383]:
active_inno_df['CMP_NM'].loc[active_inno_df['CEO_NM']=='김명구']

3             중원산업
14270    인천북항벌크터미널
Name: CMP_NM, dtype: object

In [404]:
test2name = new_inno_biz['업체명'].loc[new_inno_biz['대표자']=='최규호'].tolist()
test2name

['코팅코리아']

In [409]:
testlist1= ['홍순철',
 '김명화',
 '김명구',
 '김종만',
 '정득부',
 '아와츠시게키',
 '리와이킷애니타',
 '변재철',
 '최규호']
testlist2 = ['홍순철','김명화','김명구','김종만','최규호'] 

final=[]
for name in testlist1:
    if name in testlist2:
        # 대표자이름 중복 많음 - 여러개 회사 for문 돌리기
        test1name_list = active_inno_df['CMP_NM'].loc[active_inno_df['CEO_NM']==name].tolist()
        for company in test1name_list:
            test2name_list = new_inno_biz['업체명'].loc[new_inno_biz['대표자']==name].tolist()
            for company2 in test2name_list:
                if (company == company2) | (company2 in company):
                    final_company = active_inno_df['CMP_NM'].loc[active_inno_df['CEO_NM']==name].tolist()
                    final.append(final_company)

In [410]:
final

[['세풍폴리머', '가보테크', '세풍폴리머수지공장'], ['세풍폴리머', '가보테크', '세풍폴리머수지공장'], ['코팅코리아']]

In [185]:
# inno_active_list=[]
# for i in range(len(active_df)):
#     for j in range(len(inno_biz_df)):
#         if inno_biz_df['업체명'][i] in active_df['CMP_NM'][j]:
#             inno_active_list.append(inno_biz_df['업체명'][i])

In [ ]:
# inno_inactive_list=[]
# for i in range(len(inactive_df)):
#     for j in range(len(inno_biz_df)):
#         if inno_biz_df['업체명'][i] in inactive_df['CMP_NM'][j]:
#             inno_inactive_list.append(inno_biz_df['업체명'][i])

In [153]:
for df in [active_df, inactive_df]:
    for i in range(len(df)):
        # 조건 이름이 들어가 있는지 & 대표자명 포함인지.
        condition = (inno_biz_df['업체명'].str.contains(df['CMP_NM'][i])) & (inno_biz_df['대표자'].str.contains(df['CEO_NM'][i]))
        df['INNO_BIZ'].loc[condition]=1

IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).

In [146]:
for df in [active_df, inactive_df]:
    for i in range(len(inno_biz_df)):
        # 조건 이름이 들어가 있는지 & 대표자명 포함인지.
        condition = (df['CMP_NM'].str.contains(inno_biz_df['업체명'][i])) & (df['CEO_NM'].str.contains(inno_biz_df['대표자'][i]))
        df['INNO_BIZ'].loc[condition]=1

TypeError: 'in <string>' requires string as left operand, not Series

In [ ]:
# innerjoin으로 겹치는 기업 구하기
inno_active_innerjoin = pd.merge(active_df, inno_biz_df, left_on='CMP_NM', how='left')